In [1]:
import sys
import os
print(os.getcwd())
os.chdir("C:\\Users\\Samuel\\Dropbox\\Git Projects\\glo-7030-projet")
print(os.getcwd())

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
sys.path.append("../..")
device = "cpu"

from src.mapping.utils import warpPerspective, warpPerspectiveBatch, crop_center
from src.semantic.utils.utils import readfile

rink_2d_label = readfile('data/rink/rink_label')
rink_2d = crop_center(rink_2d_label, 450, 256)

semseg = readfile("data/train/image-104")
semseg = crop_center(semseg, 450, 256)

from src.semantic.net_parameters import p_classes_color
import matplotlib as mpl
import matplotlib.pyplot as plt
cmap = mpl.colors.ListedColormap(p_classes_color)

#'ice': 1, 'board': 2, 'circlezone': 3, 'circlemid': 4, 'goal': 5, 'blue': 6, 'red': 7, 'fo': 8

plt.subplot(1,2,1)
plt.imshow(semseg,cmap=cmap)
plt.subplot(1,2,2)
plt.imshow(rink_2d,cmap=cmap)

C:\Users\Samuel\Dropbox\Git Projects\glo-7030-projet\src\mapping
C:\Users\Samuel\Dropbox\Git Projects\glo-7030-projet


In [2]:
h = np.array([[1,0,0],[0,1,0],[0,0,1]])
proj= warpPerspective(semseg, h)
print(h)
print(proj[225,128])

[[1 0 0]
 [0 1 0]
 [0 0 1]]
1.0


In [3]:
# loss zone-face-offs
x3 = np.array([[75],[375]])
y3 = np.array([[70],[180]])
p3 = np.where(proj==3)

dx3 = np.amin((p3[0] - x3)**2, axis=0)
dy3 = np.amin((p3[1] - y3)**2, axis=0)
np.sum(dx3+dy3)/max(1,len(p3[0]))

0.0

In [ ]:
k = 8
semseg = torch.tensor(semseg)

In [33]:
#h = np.array([[1,0,0],[0,1,0],[0,0,1]])
h = torch.tensor([[[1,0,0],[0,1,0],[0,0,1]],
                  [[1,0,0],[0,1,0],[0,0,1]],
                  [[1,0,0],[0,1,0],[0,0,1]],
                  [[1,0,0],[0,1,0],[0,0,1]]], dtype=torch.float)
b = h.shape[0]

In [45]:
semseg2 = semseg.reshape(1,450,256)
semseg2 = torch.cat((semseg2,semseg2,semseg2,semseg2),0)
# semseg

#warpPerspective(semseg2[0], h[0])
#warpPerspective(semseg2[1], h[1])
#warpPerspective(semseg2[2], h[2])
#warpPerspective(semseg2[3], h[3])
projs = warpPerspectiveBatch(semseg2, h, device)

In [86]:
if k == 8:
    x = torch.tensor([[70],[180]])
    y = torch.tensor([[170],[280]])
 
for i in range(b):
    print(i)
    p = torch.where(projs[i]==k)
    dx = torch.min((p[0] - x)**2,dim=0).values
    dy = torch.min((p[1] - y)**2,dim=0).values
    loss = torch.sum(dx+dy)/max(1,len(p[0]))
    print(loss)


    

0
<class 'torch.Tensor'>
tensor(17797)
1
<class 'torch.Tensor'>
tensor(17797)
2
<class 'torch.Tensor'>
tensor(17797)
3
<class 'torch.Tensor'>
tensor(17797)


In [102]:
xx = torch.tensor([[[0],[0]],[[0],[0]],[[0],[0]],[[75],[375]],[[225],[0]],[[0],[0]],[[0],[0]],[[0],[0]],[[170],[280]]])
yy = torch.tensor([[[0],[0]],[[0],[0]],[[0],[0]],[[70],[180]],[[128],[0]],[[0],[0]],[[0],[0]],[[0],[0]],[[70],[180]]])
projs = warpPerspectiveBatch(semseg2, h, device)

In [106]:
        loss = 0
        for k in (3,4,8):
            for i in range(b):
                x = xx[k]
                y = yy[k]
                p = torch.where(projs[i]==k)
                
                if k == 4:
                    x = x[0]
                    y = y[0]
                
                dx = (p[0] - x)**2
                dy = (p[1] - y)**2

                if k == 3 | k == 8:
                    dx = torch.min(dx,dim=0).values
                    dy = torch.min(dy,dim=0).values
            
                loss = loss + torch.sum(dx+dy)/max(1,len(p[0]))
                print(loss)



tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(25989)
tensor(51978)
tensor(77967)
tensor(103956)
tensor(164526)
tensor(225096)
tensor(285666)
tensor(346236)


In [101]:
i = 3
k = 8
xx[k-1]


tensor([[170],
        [280]])

In [95]:
xx.shape

torch.Size([8, 2, 1])